In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/test.txt
/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/train.txt
/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/dev.txt


In [2]:
# load text 

filename = '/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/dev.txt'
file = open(filename, 'rt')
text = file.read()
file.close()

In [3]:
# load doc into memory
def load_doc(filename):
 # open the file as read only
 file = open(filename, mode='rt', encoding='utf-8')
 # read all text
 text = file.read()
 # close the file
 file.close()
 return text

In [4]:
# split a loaded document into sentences
def to_sentences(doc):
 return doc.strip().split('\n')

In [5]:
# shortest and longest sentence lengths
def sentence_lengths(sentences):
 lengths = [len(s.split()) for s in sentences]
 return min(lengths), max(lengths)

In [16]:
# Function to load a document
def load_doc(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return file.read()

# Function to split the document into sentences
def to_sentences(doc):
    # Placeholder implementation
    return doc.split('\n')  # Assuming each line is a sentence

# Function to find minimum and maximum lengths of sentences
def sentence_lengths(sentences):
    if not sentences:
        return 0, 0  # Return default values if sentences list is empty
    min_len = min(len(sentence.split()) for sentence in sentences)
    max_len = max(len(sentence.split()) for sentence in sentences)
    return min_len, max_len

# Load Hinglish data
filename_hinglish = '/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/dev.txt'
doc_hinglish = load_doc(filename_hinglish)
sentences_hinglish = to_sentences(doc_hinglish)
minlen_hinglish, maxlen_hinglish = sentence_lengths(sentences_hinglish)
print('Hinglish data: sentences=%d, min=%d, max=%d' % (len(sentences_hinglish), minlen_hinglish, maxlen_hinglish))

Hinglish data: sentences=1021, min=0, max=155


> creates a vocabulary for a given list of sentences.

In [24]:
from collections import Counter

# create a frequency table for all words

def to_vocab(lines):
    vocab = Counter()
    for line in lines:
        tokens = line.split()
        vocab.update(tokens)
    return vocab


In [25]:
# remove all words with a frequency below a threshold
def trim_vocab(vocab, min_occurance):
 tokens = [k for k,c in vocab.items() if c >= min_occurance]
 return set(tokens)


In [27]:
# mark all OOV with "unk" for all lines
def update_dataset(lines, vocab):
    new_lines = []
    for line in lines:
        new_tokens = []
        for token in line.split():
            if token in vocab:
                new_tokens.append(token)
            else:
                new_tokens.append('unk')
        new_line = ' '.join(new_tokens)
        new_lines.append(new_line)
    return new_lines

> The size of the English vocabulary 

In [29]:
# Load Hinglish dataset
filename = '/kaggle/input/eng-hinglish-machine-translation/Eng - Hinglish/dev.txt'
doc = load_doc(filename)
lines = to_sentences(doc)

# Calculate vocabulary
vocab = to_vocab(lines)

# Update dataset with OOV marked as "unk"
updated_lines = update_dataset(lines, vocab)


Original: Yeah , that was nice while it lasted . Seems to have dried up though	हां , वह वाक्य में अछे हैं .. लगता हैं कि वह DRIED UP किये हैं 
Updated: Yeah , that was nice while it lasted . Seems to have dried up though हां , वह वाक्य में अछे हैं .. लगता हैं कि वह DRIED UP किये हैं
Original: I live in Ohio and I hear about those " credits " for movies being made here . Normally I'd watch Hugh Jackman in other movies .	में OHIO में रहने वाला हूँ , और मैं काफी US MOVIES का CREDITS सुना हूँ ... NORMALLY में ह्यू JACKMAN का MOVIES देखना पसन्द करता 
Updated: I live in Ohio and I hear about those " credits " for movies being made here . Normally I'd watch Hugh Jackman in other movies . में OHIO में रहने वाला हूँ , और मैं काफी US MOVIES का CREDITS सुना हूँ ... NORMALLY में ह्यू JACKMAN का MOVIES देखना पसन्द करता
Original: He's a decent actor .	वह DECENT ACTOR हैं 
Updated: He's a decent actor . वह DECENT ACTOR हैं
Original: Didn't realize this movie was that old - 2011 , time goes quick . I 

In [30]:
# Example usage:
for line, updated_line in zip(lines[:5], updated_lines[:5]):
    print("Original:", line)

Original: Yeah , that was nice while it lasted . Seems to have dried up though	हां , वह वाक्य में अछे हैं .. लगता हैं कि वह DRIED UP किये हैं 
Original: I live in Ohio and I hear about those " credits " for movies being made here . Normally I'd watch Hugh Jackman in other movies .	में OHIO में रहने वाला हूँ , और मैं काफी US MOVIES का CREDITS सुना हूँ ... NORMALLY में ह्यू JACKMAN का MOVIES देखना पसन्द करता 
Original: He's a decent actor .	वह DECENT ACTOR हैं 
Original: Didn't realize this movie was that old - 2011 , time goes quick . I don't recognize the other actors .	मुझे यकीन ही नहीं होता कि वो 2011 का हैं , TIME THO QUICK जाता हैं ... मुझे दूसरे ACTORS को RECOGNIZE नहीं हुवा 
Original: Neither do i	मुझे भी 


In [31]:
print("Updated:", updated_line)

Updated: Neither do i मुझे भी


In [32]:
vocab = to_vocab(lines)
print('Hinglish Vocabulary: %d' % len(vocab))

Hinglish Vocabulary: 3981


In [33]:
vocab = trim_vocab(vocab, 5)
print('New Hinglish Vocabulary: %d' % len(vocab))

New Hinglish Vocabulary: 755


In [34]:
# spot check
for i in range(10):
 print(lines[i])

Yeah , that was nice while it lasted . Seems to have dried up though	हां , वह वाक्य में अछे हैं .. लगता हैं कि वह DRIED UP किये हैं 
I live in Ohio and I hear about those " credits " for movies being made here . Normally I'd watch Hugh Jackman in other movies .	में OHIO में रहने वाला हूँ , और मैं काफी US MOVIES का CREDITS सुना हूँ ... NORMALLY में ह्यू JACKMAN का MOVIES देखना पसन्द करता 
He's a decent actor .	वह DECENT ACTOR हैं 
Didn't realize this movie was that old - 2011 , time goes quick . I don't recognize the other actors .	मुझे यकीन ही नहीं होता कि वो 2011 का हैं , TIME THO QUICK जाता हैं ... मुझे दूसरे ACTORS को RECOGNIZE नहीं हुवा 
Neither do i	मुझे भी 
This reads like a standard boxing movie ... but with robots	में STANDARD BOXING MOVIES की तरह पड़ता था , लेकिन ROBOTS के साथ 
Do you like boxing as a sport ?	आपको sport यक boxing का पसंद हैं ? 
Nah	नहीं 
I don't like most sports though	मुझे कुछ sports पसंद नहीं हैं 
I'm reading through here looking for interesting facts about 